### Multivariate Hyperparameter GridSearchCV LSTM-RNN Sumatera Selatan

In [1]:
# pustaka untuk manipulasi data
import pandas as pd
from pandas import concat
from pandas import DataFrame
from pandas import read_csv
from pandas import read_excel
import numpy as np
from numpy import concatenate

# pustaka untuk waktu komputasi
import time
from datetime import datetime

# Pustaka untuk visualisasi data
import seaborn as sns # Visualization
from matplotlib import pyplot
from matplotlib import pyplot as plt

# Pustaka untuk membuat data latih dan data uji.
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# Pustaka untuk membuat model prediksi LSTM-RNN
import itertools
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.optimizers import Adam, Adamax, RMSprop, SGD

# Early stoping
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# Pustaka untuk  evaluasi model prediksi
from math import sqrt
from sklearn.metrics import mean_squared_error

In [2]:
# Set waktu komputasi
start = time.time()

In [3]:
# fix random seed for reproducibility
np.random.seed(1)

In [4]:
# membaca dataset
dataset = pd.read_excel("D:/Belajar Python/dataset/dataset tesis/dataset.xlsx")

In [5]:
# set index tanggal
dataset = dataset.set_index("tanggal")

In [6]:
# set informasi dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 240 entries, 2001-01-31 to 2020-12-31
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sst             240 non-null    float64
 1   soi             240 non-null    float64
 2   oni             240 non-null    float64
 3   hotspot_riau    240 non-null    int64  
 4   hotspot_sumsel  240 non-null    int64  
dtypes: float64(3), int64(2)
memory usage: 11.2 KB


In [7]:
# menampilkan dataset
print(dataset.head())

             sst  soi   oni  hotspot_riau  hotspot_sumsel
tanggal                                                  
2001-01-31 -0.83  1.6 -0.68            17               3
2001-02-28 -0.61  2.8 -0.52           159               5
2001-03-31 -0.38  1.5 -0.44           479              10
2001-04-30 -0.26  0.3 -0.34            45              20
2001-05-31 -0.25 -0.8 -0.25            81              39


### Studi kasus wilayah Sumatra Selatan

In [8]:
# memilih area studi
df_sumsel = dataset[["hotspot_sumsel", "sst", "soi", "oni"]]
df_sumsel.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 240 entries, 2001-01-31 to 2020-12-31
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   hotspot_sumsel  240 non-null    int64  
 1   sst             240 non-null    float64
 2   soi             240 non-null    float64
 3   oni             240 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 9.4 KB


In [9]:
print(df_sumsel.head())

            hotspot_sumsel   sst  soi   oni
tanggal                                    
2001-01-31               3 -0.83  1.6 -0.68
2001-02-28               5 -0.61  2.8 -0.52
2001-03-31              10 -0.38  1.5 -0.44
2001-04-30              20 -0.26  0.3 -0.34
2001-05-31              39 -0.25 -0.8 -0.25


In [10]:
# ensure all data is float
values = df_sumsel.values
values = values.astype('float32')

### Normalisasi Data

In [11]:
# membuat fungsi max-min scaler
# menggunakan pustaka scikit-learn
def normalisasi_max_min(df):
    # memanggil fungsi max min scaler
    hasil = preprocessing.MinMaxScaler(feature_range=(-1, 1))
    
    # proses max min scaler
    hasil = hasil.fit_transform(df)
    
    # pengembalian nilai
    return hasil

In [12]:
# normalize features
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled = scaler.fit_transform(values)

In [13]:
np.round(scaled[:5],6)

array([[-0.999723, -0.561983,  0.36    , -0.551402],
       [-0.999445, -0.471074,  0.6     , -0.476636],
       [-0.998751, -0.376033,  0.34    , -0.439252],
       [-0.997364, -0.326446,  0.1     , -0.392523],
       [-0.994728, -0.322314, -0.12    , -0.350467]], dtype=float32)

### Normalisasi dan Supervised learning

In [14]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    
    # return value
    return agg

In [15]:
# supervised learning
reframed = series_to_supervised(scaled, 1, 1)

In [16]:
# drop columns we don't want to predict
reframed.drop(reframed.columns[[5,6,7]], axis=1, inplace=True)

In [17]:
values = reframed.values

In [18]:
print(reframed.head())

   var1(t-1)  var2(t-1)  var3(t-1)  var4(t-1)   var1(t)
1  -0.999723  -0.561983       0.36  -0.551402 -0.999445
2  -0.999445  -0.471074       0.60  -0.476636 -0.998751
3  -0.998751  -0.376033       0.34  -0.439252 -0.997364
4  -0.997364  -0.326446       0.10  -0.392523 -0.994728
5  -0.994728  -0.322314      -0.12  -0.350467 -0.999029


### Data latih dan Data uji

In [19]:
# split into train and test sets
train_size = int(len(values) * 0.8)
test_size = len(values) - train_size
train, test = values[0:train_size,:], values[train_size:len(values),:]

In [20]:
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

In [21]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

In [22]:
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(191, 1, 4) (191,) (48, 1, 4) (48,)


### Hyperparameter GridSearchCV LTSM-RNN

In [23]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [24]:
parameters = {'neurons' : [10],
              'activation' : ['sigmoid', 'tanh', 'relu', 'selu', 'elu', 'softplus'],
              'optimizer' : ['adam', 'adamax', 'rmsprop', 'sgd'],
              'dropout_rate' : [0.1, 0.2, 0.3, 0.4, 0.5],
              'epochs' : [2000],
              'batch_size' : [8, 16, 32, 64],
              'verbose' : [0]}

In [25]:
keys = parameters.keys()
values = (parameters[key] for key in keys)
combinations = [dict(zip(keys, combination)) for combination in itertools.product(*values)]
print(len(combinations), str('kombinasi hyperparameter'))

480 kombinasi hyperparameter


In [26]:
def build_classifier(neurons='', activation='', optimizer='', dropout_rate=''):
    tf.keras.backend.clear_session()
    
    # design network
    grid_model = Sequential()
    grid_model.add(LSTM(units=neurons, activation=activation, input_shape=(train_X.shape[1], train_X.shape[2])))
    grid_model.add(Dropout(dropout_rate))
    grid_model.add(Dense(1))

    # model compile
    grid_model.compile(loss='mae', optimizer=optimizer)
    
    # return value
    return grid_model

In [27]:
grid_model = KerasRegressor(build_fn=build_classifier)

In [28]:
grid_search = GridSearchCV(estimator=grid_model, param_grid=parameters, n_jobs=-1, cv=2, scoring= 'neg_root_mean_squared_error')

In [29]:
grid_result = grid_search.fit(train_X, train_y)

In [30]:
# summarize results
print("Best parameters: %f using %s\n" % (grid_search.best_score_, grid_search.best_params_))

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best parameters: -0.203195 using {'activation': 'tanh', 'batch_size': 16, 'dropout_rate': 0.1, 'epochs': 2000, 'neurons': 10, 'optimizer': 'adam', 'verbose': 0}

-0.226979 (0.058035) with: {'activation': 'sigmoid', 'batch_size': 8, 'dropout_rate': 0.1, 'epochs': 2000, 'neurons': 10, 'optimizer': 'adam', 'verbose': 0}
-0.249181 (0.058202) with: {'activation': 'sigmoid', 'batch_size': 8, 'dropout_rate': 0.1, 'epochs': 2000, 'neurons': 10, 'optimizer': 'adamax', 'verbose': 0}
-0.220630 (0.051664) with: {'activation': 'sigmoid', 'batch_size': 8, 'dropout_rate': 0.1, 'epochs': 2000, 'neurons': 10, 'optimizer': 'rmsprop', 'verbose': 0}
-0.268517 (0.061958) with: {'activation': 'sigmoid', 'batch_size': 8, 'dropout_rate': 0.1, 'epochs': 2000, 'neurons': 10, 'optimizer': 'sgd', 'verbose': 0}
-0.225168 (0.054898) with: {'activation': 'sigmoid', 'batch_size': 8, 'dropout_rate': 0.2, 'epochs': 2000, 'neurons': 10, 'optimizer': 'adam', 'verbose': 0}
-0.257483 (0.062022) with: {'activation': 'sigmoi

In [31]:
result = pd.concat([pd.DataFrame(grid_search.cv_results_["params"]),pd.DataFrame(grid_search.cv_results_["mean_test_score"], columns=["score"])],axis=1)

In [32]:
result.sort_values(by="score", ascending=False)

,activation,batch_size,dropout_rate,epochs,neurons,optimizer,verbose,score
100,tanh,16,0.1,2000,10,adam,0,-0.203195
264,selu,16,0.2,2000,10,adam,0,-0.203926
340,elu,16,0.1,2000,10,adam,0,-0.205850
260,selu,16,0.1,2000,10,adam,0,-0.205903
292,selu,32,0.4,2000,10,adam,0,-0.206018
...,...,...,...,...,...,...,...,...
172,relu,8,0.4,2000,10,adam,0,-0.331005
256,selu,8,0.5,2000,10,adam,0,-0.333821
200,relu,32,0.1,2000,10,adam,0,-0.336587
212,relu,32,0.4,2000,10,adam,0,-0.337786


In [33]:
# result.sort_values(by="score", ascending=False).to_csv('multivariate_gridsearch_lstm.csv',index=False)

### Evaluasi Model LSTM-RNN

In [34]:
# Set akhir waktu komputasi 
end = time.time()

In [35]:
# Proses menghitung waktu komputasi
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)

In [36]:
# Hasil waktu komputasi
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

00:34:06.86
